```
Copyright 2023 ServiceNow
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

This notebook takes the results stored in `./data/ror_results.csv` and create the colour plots for a single experiment or a single metric.

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
import glob

In [ ]:
from ror.experiments import h0_vs_h1 as exp_def

In [ ]:
full_df = pd.read_csv("./data/ror_results.csv")
groupby_dfs = {index: df for index, df in full_df.groupby(["experiment", "metric", "dim", "num_samples"])}

In [ ]:
def get_beta(exp_name, metric_name, dim, num_samples):
    if (exp_name, metric_name, dim, num_samples) not in groupby_dfs:
        return np.nan
    df = groupby_dfs[exp_name, metric_name, dim, num_samples]

    diffs = df.GT - df.FCST
    d_mean = diffs.mean()
    d_std = diffs.std()
    
    alpha = 0.05
    threshold = exp_def.NUM_DRAWS**0.5 * d_std * scipy.stats.norm.ppf(exp_def.ALPHA)
    return 1 - scipy.stats.norm.cdf((threshold - exp_def.NUM_DRAWS * d_mean) / (exp_def.NUM_DRAWS**0.5 * d_std))

In [ ]:
def plot_exp_metric(exp_name, metric_name):
    betas = np.zeros((len(exp_def.NUM_SAMPLES_LIST), len(exp_def.DIM_LIST)))
    for i in range(len(exp_def.DIM_LIST)):
        for j in range(len(exp_def.NUM_SAMPLES_LIST)): 
            betas[j][i] = get_beta(exp_name, metric_name, exp_def.DIM_LIST[i], exp_def.NUM_SAMPLES_LIST[j])
    
    plt.figure(figsize=(12,8))
    plt.pcolor(
        exp_def.DIM_LIST,
        exp_def.NUM_SAMPLES_LIST,
        betas,
        shading="nearest",
        cmap="viridis_r",
        norm=colors.BoundaryNorm(boundaries=np.linspace(0, 1, 21), ncolors=256),
    )
    
    cb = plt.colorbar(
        label=f"Probabilities of false negative with {exp_def.NUM_DRAWS} Ground-Truth samples",
        ticks=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
    )
    cb.ax.plot([0, 1], [exp_def.BETA_NLL]*2, color='w', linewidth=3)  # Add a notes to where the Log-likelihood lies
    cb.ax.set_yticklabels(["0.0", "0.2 = NLL", "0.4", "0.6", "0.8", "1.0"]) 
    
    plt.title(f"Metric = {metric_name}, Distributions = {exp_name}")
    plt.xlabel("dimension")
    plt.ylabel("Number of Forecasting samples")
    
    plt.gca().set_xscale('log', base=2)
    plt.gca().set_yscale('log', base=2)
    plt.xlim(exp_def.DIM_LIST[0] / 2**0.5, exp_def.DIM_LIST[-1] * 2**0.5)
    plt.ylim(exp_def.NUM_SAMPLES_LIST[0] / 2**0.5, exp_def.NUM_SAMPLES_LIST[-1] * 2**0.5)
    
    plt.show()

In [ ]:
%%time
for metric in exp_def.METRIC_FUNCTIONS.keys():
    plot_exp_metric("wrong_mean_single", metric)

In [ ]:
for exp_name in exp_def.EXP_LIST:
    plot_exp_metric(exp_name, "crps_quantile")